#### 安裝套件

In [1]:
 ! python --version

Python 3.11.13


In [2]:
!pip install datasets evaluate jiwer librosa
!pip install --upgrade bitsandbytes transformers==4.50.0 accelerate
!pip install ctranslate2==4.4.0 whisperx
!apt-get install libcudnn8 libcudnn8-dev

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5

#### 載入函式庫與參數設定

In [3]:
import os
import pandas as pd
import torch
import numpy as np
from datasets import Dataset, Audio
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
from tqdm import tqdm
from transformers import EarlyStoppingCallback
from ctranslate2.converters import TransformersConverter
import whisperx
from transformers import WhisperProcessor, WhisperTokenizer
import json
import zipfile
from jiwer import wer

In [4]:
def setup_directories(version: str = "v99"):
    # Create result directory and version subdirectory
    result_dir = "model_result"
    result_dir = os.path.join(result_dir, "task1")
    version_dir = os.path.join(result_dir, version)
    model_dir = os.path.join(version_dir, "model")

    # Create directories if they don't exist
    os.makedirs(result_dir, exist_ok=True)
    os.makedirs(version_dir, exist_ok=True)
    os.makedirs(model_dir, exist_ok=True)
    print(result_dir, version_dir, model_dir)
    return version_dir, model_dir

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Setup directories
version = "v1"
version_dir, model_dir = setup_directories(version)


# Check CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# 設定模型路徑
final_model_path = os.path.join(model_dir, "fine-tuning-whisper")
final_ct2_model_path = os.path.join(model_dir, "fine-tuning-whisper-ct2")
print(f"Model saved to {final_model_path}")

# load fine-tuning whisper
zip_model_path = "/content/drive/MyDrive/AICUP/fine-tuning-whisper.zip"
extract_model_path = "/content/drive/MyDrive/AICUP/model_result/task1/v1/model/fine-tuning-whisper"
with zipfile.ZipFile(zip_model_path, 'r') as zip_ref:
  zip_ref.extractall(extract_model_path)

model_result/task1 model_result/task1/v1 model_result/task1/v1/model
Using device: cuda
GPU: Tesla T4
Model saved to model_result/task1/v1/model/fine-tuning-whisper


#### 資料前處理

In [6]:
def prepare_dataset(validation_path: str): # train_path: str,

    val_audio_files = [f for f in os.listdir(validation_path) if f.endswith('.wav')]
    val_dict = {
        "audio": [os.path.join(validation_path, audio_file) for audio_file in val_audio_files],
        "text": ["" for _ in val_audio_files]  # 空字串作為預設文字
    }
    val_dataset = Dataset.from_dict(val_dict)
    val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000))
    # 回傳兩個資料集
    return {
        "val": val_dataset
    }

In [7]:
def prepare_features(batch, processor):
    # Process audio
    audio = batch["audio"]

    # Compute log-mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"],
        language="en",   # or "zh"
        return_tensors="pt"
    ).input_features[0]

    # Encode target text to label ids 並限制最大長度為 448
    batch["labels"] = processor.tokenizer(
        batch["text"],
        max_length=448,
        truncation=True,
        padding="max_length",

    ).input_ids
    batch["file_path"] = batch["audio"]["path"]
    batch["audio_file_name"] = os.path.splitext(os.path.basename(batch["audio"]["path"]))[0]

    return batch

In [8]:
zip_path = "/content/drive/MyDrive/AICUP/Validation_Dataset.zip"
extract_dir = "/content/audio_dataset"
os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extract_dir)

audio_files = []
for root, dirs, files in os.walk(extract_dir):
  for file in files:
    if file.lower().endswith('.wav'):
      audio_files.append(os.path.join(root, file))

print(f"found {len(audio_files)} audio files")

found 710 audio files


In [11]:
# Prepare dataset
processor = WhisperProcessor.from_pretrained("/content/drive/MyDrive/AICUP/model_result/task1/v1/model/fine-tuning-whisper/fine-tuning-whisper")
# "audio_dataset/sample/Training_Dataset",
dataset = prepare_dataset("/content/audio_dataset/audio")
# Process datasets
processed_dataset = {}
for split in ["val"]: # "train", "test",
    print(f"Processing {split} dataset...")
    processed_dataset[split] = dataset[split].map(
        lambda x: prepare_features(x, processor=processor),
        remove_columns= dataset[split].column_names,
        num_proc=1
    )

Processing val dataset...


Map:   0%|          | 0/710 [00:00<?, ? examples/s]

#### 進行轉檔 Huggingface to Ctranslate2

In [12]:
def convert_hf_model_to_ct2(model_name_or_path: str, output_dir: str, quantization: str = "float32", trust_remote_code: bool = True, device: str = "cuda"):
    """
    Hugging Face convert to  CTranslate2

    parameter:
        model_name_or_path (str): Hugging Face moedel name or local directory.
        output_dir (str): Output directory where the CTranslate2 model is saved.
        quantization (str): Weight quantization scheme (possible values are: int8, int8_float32, int8_float16, int8_bfloat16, int16, float16, bfloat16, float32).
        trust_remote_code (bool): Allow converting models using custom code.
    """
    # 初始化轉換器
    converter = TransformersConverter(model_name_or_path,copy_files=["preprocessor_config.json", "tokenizer.json"] , trust_remote_code=trust_remote_code)
    # 執行轉換
    converter.convert(output_dir=output_dir,force=True)
    model = whisperx.load_model(output_dir, device=device)
    return model

In [13]:
print("Converting model to CTranslate2 format...")
final_model_path = "/content/drive/MyDrive/AICUP/model_result/task1/v1/model/fine-tuning-whisper/fine-tuning-whisper"
model = convert_hf_model_to_ct2(
    model_name_or_path=final_model_path,
    output_dir=final_ct2_model_path,
    # quantization="float16",
    trust_remote_code=True
)

Converting model to CTranslate2 format...


DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


#### WhisperX進行轉錄

In [ ]:
# Save predictions to TSV file  VALIDATION
def save_predictions_to_tsv(predictions,filenames, version_dir,export_json_file):
    task1_output_file = os.path.join(version_dir, "task1_answer.txt")
    output_file = os.path.join(version_dir, "val_results.txt")
    print("Saving predictions to file...")
    json_output_file = os.path.join(version_dir, "task1_answer_timestamps.json")
    with open(json_output_file, "w", encoding="utf-8") as f:
        json.dump(export_json_file, f, ensure_ascii=False, indent=2)
    print(f"Results saved to {task1_output_file}")

In [15]:
def calculate_output(model, eval_dataloader, version_dir):
    print("Calculating ouput score...")
    predictions = []
    filenames = []
    export_json_file = {}

    i = 0
    for batch in tqdm(eval_dataloader, desc="處理評估資料", unit="batch"):
        with torch.no_grad():

            # Move batch to device
            audio = batch["file_path"]

            result = model.transcribe(audio, batch_size=1)
            segments = result.get("segments", [])

            if not segments:
                print(f"Warning: No segments found for audio file: {batch['audio_file_name']}")
                pred_str = ""
                transcript_dict = {
                    "language": result.get("language", "unknown"),
                    "segments": []
                }
                word_segments = []
                print('no audio data.')
            else:

                transcript_dict = {
                    "language": result.get("language", "unknown"),
                    "segments": []
                }
                align_model, metadata = whisperx.load_align_model(
                    language_code=result.get("language", "en"),
                    device="cuda"
                )

                alignment = whisperx.align(segments, align_model, metadata, audio, device="cuda")
                word_segments = alignment["word_segments"]


                for seg in segments:

                    words_in_seg = [
                        {
                            "word": w["word"],
                            "start": w["start"],
                            "end": w["end"],
                            "probability": w.get("probability", None)
                        }
                        for w in word_segments
                        if w["start"] >= seg["start"] and w["end"] <= seg["end"]+1
                    ]
                    transcript_dict["segments"].append({
                        "text": seg.get("text", ""),
                        "start": seg.get("start", 0.0),
                        "end": seg.get("end", 0.0),
                        "words": words_in_seg
                    })

                # Decode predictions
                # 訪問第一個 segment 的 text，並使用 .get()
                pred_str = segments[0].get("text", "")


            export_json_file[batch["audio_file_name"]] = transcript_dict

            predictions.extend([pred_str])
            filenames.extend([batch["audio_file_name"]])
            i += 1


    # Save predictions to file
    save_predictions_to_tsv(predictions,filenames, version_dir,export_json_file)

    return 0

In [16]:
calculate_output(model, processed_dataset["val"], version_dir)

Calculating ouput score...


處理評估資料:   0%|          | 0/710 [00:00<?, ?batch/s]/usr/local/lib/python3.11/dist-packages/pyannote/audio/utils/reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


Detected language: en (1.00) in first 30s of audio...


Downloading: "https://download.pytorch.org/torchaudio/models/wav2vec2_fairseq_base_ls960_asr_ls960.pth" to /root/.cache/torch/hub/checkpoints/wav2vec2_fairseq_base_ls960_asr_ls960.pth

  0%|          | 0.00/360M [00:00<?, ?B/s]
  0%|          | 1.12M/360M [00:00<00:32, 11.6MB/s]
  1%|▏         | 4.62M/360M [00:00<00:14, 26.1MB/s]
  4%|▍         | 15.4M/360M [00:00<00:05, 65.2MB/s]
 11%|█         | 38.8M/360M [00:00<00:02, 136MB/s] 
 20%|█▉        | 71.6M/360M [00:00<00:01, 210MB/s]
 26%|██▌       | 93.0M/360M [00:00<00:01, 215MB/s]
 33%|███▎      | 120M/360M [00:00<00:01, 239MB/s] 
 42%|████▏     | 151M/360M [00:00<00:00, 263MB/s]
 50%|████▉     | 179M/360M [00:00<00:00, 269MB/s]
 59%|█████▉    | 212M/360M [00:01<00:00, 294MB/s]
 67%|██████▋   | 240M/360M [00:01<00:00, 291MB/s]
 75%|███████▍  | 270M/360M [00:01<00:00, 290MB/s]
 83%|████████▎ | 299M/360M [00:01<00:00, 293MB/s]
100%|██████████| 360M/360M [00:01<00:00, 245MB/s]
處理評估資料:   0%|          | 1/710 [00:09<1:50:58,  9.39s/batch]

Detected language: en (1.00) in first 30s of audio...


...

處理評估資料: 100%|█████████▉| 709/710 [37:26<00:03,  3.28s/batch]

Detected language: en (1.00) in first 30s of audio...


處理評估資料: 100%|██████████| 710/710 [37:28<00:00,  3.17s/batch]


Saving predictions to file...
Results saved to model_result/task1/v1/task1_answer.txt


0